In [1]:
import cx_Oracle


In [ ]:

import pandas as pd

# Read CSV
df = pd.read_csv("../data/processed/enhanced_sentiment_analysis.csv")

conn = cx_Oracle.connect("bank_reviews/12345678@localhost:1521/XEPDB1")
cur = conn.cursor()

# Insert banks and build bank name → id mapping
bank_ids = {}
banks = df["bank"].unique()

for bank_name in banks:
    cur.execute("MERGE INTO banks b USING (SELECT :1 name FROM dual) d ON (b.name = d.name) WHEN NOT MATCHED THEN INSERT (name) VALUES (:1)", [bank_name])
    conn.commit()
    cur.execute("SELECT bank_id FROM banks WHERE name = :1", [bank_name])
    bank_ids[bank_name] = cur.fetchone()[0]

# Insert reviews
for _, row in df.iterrows():
    # Handle NaN values for numeric and string columns
    review_id = int(row["review_id"]) if pd.notnull(row["review_id"]) else None
    bank_id = bank_ids[row["bank"]]
    review_text = row["review_text"] if pd.notnull(row["review_text"]) else None
    sentiment_label = row["sentiment_label"] if pd.notnull(row["sentiment_label"]) else None
    sentiment_score = float(row["sentiment_score"]) if pd.notnull(row["sentiment_score"]) else None
    identified_themes = row["identified_theme(s)"] if pd.notnull(row["identified_theme(s)"]) else None
    rating = float(row["rating"]) if pd.notnull(row["rating"]) else None

    cur.execute("""
        INSERT INTO reviews (
            review_id, bank_id, review_text, sentiment_label, sentiment_score,
            identified_themes, rating
        ) VALUES (:1, :2, :3, :4, :5, :6, :7)
    """, (
        review_id,
        bank_id,
        review_text,
        sentiment_label,
        sentiment_score,
        identified_themes,
        rating
    ))

conn.commit()
cur.close()
conn.close()


In [ ]:

# Connect to Oracle
conn = cx_Oracle.connect("bank_reviews/12345678@localhost:1521/XEPDB1")
cur = conn.cursor()

# Tables to export
tables = ["BANKS", "REVIEWS"]

# Open file to write dump
with open("../dumps/bank_reviews_dump.sql", "w", encoding="utf-8") as f:

    # Step 1: Export schema (DDL)
    for table in tables:
        cur.execute(f"""
            SELECT DBMS_METADATA.GET_DDL('TABLE', '{table}') FROM dual
        """)
        ddl = cur.fetchone()[0].read()  # CLOB to string
        f.write(f"-- DDL for {table}\n{ddl};\n\n")

    # Step 2: Export data (INSERT statements)
    for table in tables:
        cur.execute(f"SELECT * FROM {table}")
        rows = cur.fetchall()
        columns = [col[0] for col in cur.description]

        f.write(f"-- Data for {table}\n")
        for row in rows:
            values = []
            for val in row:
                if val is None:
                    values.append("NULL")
                elif isinstance(val, str):
                    values.append(f"'{val.replace("'", "''")}'")
                else:
                    values.append(str(val))
            col_str = ", ".join(columns)
            val_str = ", ".join(values)
            f.write(f"INSERT INTO {table} ({col_str}) VALUES ({val_str});\n")
        f.write("\n")

cur.close()
conn.close()
print("✅ SQL dump created as bank_reviews_dump.sql")


✅ SQL dump created as bank_reviews_dump.sql
